In [1]:
import os
os.chdir('..')

In [2]:
import ephem
from datetime import datetime, date, timedelta
import pytz
from dateutil import tz
import numpy as np

In [3]:
latitude = 37.9543963
longitude = -91.7831068
NOON = 12
curr_year = 2023

In [4]:
from DataManager.WorldClim import WorldClimAPI
wc_api = WorldClimAPI()

### Functional Code

In [5]:
def daytime(date, obs):
    utc_dt = date.astimezone(pytz.utc)
    obs.date = utc_dt
    
    sun = ephem.Sun()
    sun.compute(obs)
    
    sun_rise = obs.previous_rising(sun).datetime()
    sun_set = obs.next_setting(sun).datetime()
    
    diff_sec = (sun_set - sun_rise).seconds
    
    return diff_sec

In [6]:
def month_daytime(month, lat, lon, year):
    obs = ephem.Observer()
    obs.lat = str(lat)
    obs.lon = str(lon)
    
    day = datetime(year, month, 1, NOON)
    
    days_in_month = 0
    total_seconds = 0
    
    while(day.month == month):
        days_in_month += 1
        total_seconds += daytime(day, obs)
        day += timedelta(days=1)
    
    avg_daytime = total_seconds // days_in_month
    return avg_daytime

In [7]:
def year_daytimes(lat, lon):
    year = datetime.now().year
    results = [month_daytime(m, lat, lon, year) for m in range(1,13)]
    results = np.array(results).reshape(-1,1)
    return results

In [8]:
daytimes = year_daytimes(latitude, longitude)

### Integration With WorldClim

In [9]:
wc_df = wc_api.get_dataframe(latitude, longitude)

In [10]:
daytimes = year_daytimes(latitude, longitude)

In [11]:
wc_df['daytime'] = daytimes

In [12]:
wc_df

,elev,row,col,index,tmin,prec,wind,vapr,tmax,srad,tavg,daytime
0,315.0,312.0,529.0,674449.0,20.117750,51.0,3.82200,0.403825,40.392950,7054.0,30.254900,35608
1,315.0,312.0,529.0,674449.0,24.501200,56.0,3.84025,0.473150,46.372550,9823.0,35.435300,38909
2,315.0,312.0,529.0,674449.0,33.362600,95.0,4.17375,0.645650,56.698701,13698.0,45.028400,43121
3,315.0,312.0,529.0,674449.0,43.044800,102.0,4.21350,0.906825,67.602649,17537.0,55.329349,47531
4,315.0,312.0,529.0,674449.0,52.403450,121.0,3.32225,1.402275,75.799850,20449.0,64.096698,51262
5,315.0,312.0,529.0,674449.0,61.426849,104.0,2.98575,1.939625,83.342301,22488.0,72.389749,53146
6,315.0,312.0,529.0,674449.0,66.247701,99.0,2.69775,2.235400,88.836800,22926.0,77.545849,52211
7,315.0,312.0,529.0,674449.0,64.386501,94.0,2.56200,2.107925,87.648351,20453.0,76.017201,48954
8,315.0,312.0,529.0,674449.0,56.267151,97.0,2.71725,1.597000,79.924100,16648.0,68.098101,44742
9,315.0,312.0,529.0,674449.0,44.590100,90.0,3.15775,1.034100,69.486800,12106.0,57.036201,40364


### Test DataPreprocessor

In [5]:
from SolarIrrad.preprocessing.charlie import DataPreprocessorCharlie
from SolarIrrad.models.charlie import Charlie
from SolarIrrad.models.model import SolarPanel

In [6]:
wc_df = wc_api.get_dataframe(latitude, longitude)

In [7]:
sp = SolarPanel(0.2, 10, 0.75, 300)

In [8]:
dp = DataPreprocessorCharlie(wc_df, latitude, longitude, sp)

In [9]:
formatted = dp.format()

In [10]:
formatted

,elev,index,col,row,tmax,srad,vapr,tmin,prec,tavg,wind,daytime,adjusted_srad
0,315.0,674449.0,529.0,312.0,40.392950,7054.0,0.403825,20.117750,51.0,30.254900,3.82200,35608,2.389708
1,315.0,674449.0,529.0,312.0,46.372550,9823.0,0.473150,24.501200,56.0,35.435300,3.84025,38909,2.766255
2,315.0,674449.0,529.0,312.0,56.698701,13698.0,0.645650,33.362600,95.0,45.028400,4.17375,43121,3.184654
3,315.0,674449.0,529.0,312.0,67.602649,17537.0,0.906825,43.044800,102.0,55.329349,4.21350,47531,3.577924
4,315.0,674449.0,529.0,312.0,75.799850,20449.0,1.402275,52.403450,121.0,64.096698,3.32225,51262,3.891978
5,315.0,674449.0,529.0,312.0,83.342301,22488.0,1.939625,61.426849,104.0,72.389749,2.98575,53146,4.059036
6,315.0,674449.0,529.0,312.0,88.836800,22926.0,2.235400,66.247701,99.0,77.545849,2.69775,52211,4.001657
7,315.0,674449.0,529.0,312.0,87.648351,20453.0,2.107925,64.386501,94.0,76.017201,2.56200,48954,3.734234
8,315.0,674449.0,529.0,312.0,79.924100,16648.0,1.597000,56.267151,97.0,68.098101,2.71725,44742,3.371244
9,315.0,674449.0,529.0,312.0,69.486800,12106.0,1.034100,44.590100,90.0,57.036201,3.15775,40364,2.956107


### Mathematical Integration

In [9]:
charlie = Charlie(latitude, longitude, sp)

In [10]:
charlie.calc_month(5)

3.8919778166950794